In [1]:
import numpy as np
from numpy.fft import fft2, ifft2, fftshift, ifftshift
import matplotlib.pyplot as plt
from PIL import Image
import ipywidgets as widgets
from ipywidgets import fixed, interact_manual

In [2]:
# discrete Fourier transform
def DFT(img):
    return fftshift(fft2(img))


# take the maginitude of the result of DFT and convert it to log scale
def scale_spectrum(dft, style='mag'):
    if style == 'mag':
        mag = np.abs(dft)
        return np.log10(mag + 1)
    else:
        phase = np.angle(dft)
        return phase


# inverse discrete Fourier transform
def IDFT(dft):
    return ifft2(ifftshift(dft))

# generate Gaussian kernel
def Gaussian_kernel(size, sigma):
    gaussian_1d = scipy.signal.gaussian(size, sigma)
    return np.outer(gaussian_1d , gaussian_1d)

# helper display function
def show_DFT(dft, style='mag'):
    plt.imshow(scale_spectrum(dft, style), cmap='gray')

# low pass filter    
def low_pass(dft, sigma=15):
    size = dft.shape[0]
    gaussian = Gaussian_kernel(size, sigma)
    weight = gaussian
    return dft * weight

# high pass filter 
def high_pass(dft, sigma=15):
    size = dft.shape[0]
    gaussian = Gaussian_kernel(size, sigma)
    weight = (1 - gaussian)
    return dft * weight

In [4]:
img = np.array(Image.open('grace.png').convert('L'))
dft = DFT(img)

In [5]:
from math import ceil
from mpl_toolkits.axes_grid1 import ImageGrid
%config InlineBackend.print_figure_kwargs = {'bbox_inches':None}

def plot(step):
    dft_flat = dft.flatten()
    mag = np.abs(dft_flat)
    idx = np.argsort(mag)
    imgs = []
    step += 1
    num_cols = 10
    num_rows = ceil(step / num_cols)
    num_cols = min(step, num_cols)
    fig = plt.figure(figsize=(50, 50))
    ax = ImageGrid(fig, 111,  # similar to subplot(111)
                 nrows_ncols=(num_rows, num_cols),  # creates 2x2 grid of axes
                 axes_pad=0.5,  # pad between axes in inch.
                 )
    
    #fig, ax = plt.subplots(num_rows, num_cols)
    #fig.figsize = (10,10)
    for i in range(num_rows * num_cols):
        if i < step - 1:
            mask = np.zeros_like(mag)
            mask[idx[-(i*2+2)]] = 1
            dft_new = dft_flat * mask
            dft_new = dft_new.reshape(dft.shape)
            img = IDFT(dft_new).real
            ax[i].imshow(img, cmap='gray')
            ax[i].axis('off')
            imgs.append(img)
        elif i == step - 1:
            img_approx = np.array(imgs).sum(axis=0)
            ax[i].imshow(img_approx, cmap='gray')
            ax[i].axis('off')
        else:
            ax[i].remove()
    plt.show()
interact_manual(plot, step=(1,65))

interactive(children=(IntSlider(value=33, description='step', max=65, min=1), Button(description='Run Interact…

<function __main__.plot(step)>